# Langchain agent example - Upload research
#### Set path context

In [1]:
## Set path context
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join('../')) # or the path to your source code
sys.path.insert(0, module_path)
print(sys.path)

['/workspaces/genai-ws/genai_workshop', '/workspaces/genai-ws/genai_workshop/example_notebook', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages']


## Import dependencies 

In [2]:

from langchain.llms import OpenAI
from langchain.agents import Tool, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import DuckDuckGoSearchRun

from langchain import OpenAI, LLMMathChain
from langchain.chains import LLMChain
from langchain.agents.agent_types import AgentType
from prompt_utils.openai_utils import get_openai_chat_model

### Search tool

In [3]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

### Mock function for uploading research

In [4]:
import json


def upload_details(data_from_search: str) :#-> str:
   
   research =   f"""Uploading following details :  
             
              {data_from_search}
              
               Data uploaded.
                    """
   print(research)
   
   #return research

### Initialize tools and agents

In [5]:
llm_model = ChatOpenAI(model_name="gpt-4", temperature=0)

# Search Tool
search_tool = Tool(
    name="Search",
    func=search.run,
    description="A useful tool for searching the Internet to find information on world events, issues, dates, years, etc. Worth using for general topics. Use precise questions.",
)

# Upload Tool
upload_tool = Tool(
    name="UploadDetails",
    func=upload_details,
    description="Use this tool for uploading details on blog"
)


agent = initialize_agent(
    tools = [upload_tool,search_tool],
    llm=llm_model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,  
    handle_parsing_errors=True
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


### Agent template

In [6]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

UploadDetails: Use this tool for uploading details on blog
Search: A useful tool for searching the Internet to find information on world events, issues, dates, years, etc. Worth using for general topics. Use precise questions.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [UploadDetails, Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


#### Redefine the agent prompt

In [7]:

redefined_prompt = """
Answer the following questions as best you can. You have access to the following tools:

UploadDetails: Use this tool for uploading details on the blog
Search: A useful tool for searching the Internet to find information

Follow these steps - 
Note- if you fail in satisfying any of the step below, just move to next one.
1) Use Search tool to search details on internet. Output- data
2) Summarize the output in 100 words and use it as input for UploadDetails tool.
3) Use UploadDetails tool to upload summarized details on blog and confirms that the data has been uploaded.
4) Steps complete.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [UploadDetails, Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
    """

    


In [8]:
agent.agent.llm_chain.prompt.template = redefined_prompt

### Run query against agent

In [9]:
#query = "Seach about latest on generative AI on internet. Summarize these details in 100 words and upload these details on blog."
query = "Seach latest on generative AI, summarize and upload on blog. Once done with upload respond back saying that task is done"
agent.run(query)



> Entering new AgentExecutor chain...
I need to search for the latest information on generative AI, summarize it, and then upload it to the blog. 

Action: Search
Action Input: Latest on generative AI
Observation: (24 pages) The latest annual McKinsey Global Survey on the current state of AI confirms the explosive growth of generative AI (gen AI) tools. Less than a year after many of these tools debuted, one-third of our survey respondents say their organizations are using gen AI regularly in at least one business function. Artificial intelligence What's next for AI in 2024 Our writers look at the four hot trends to watch out for this year By Melissa Heikkilä Will Douglas Heaven January 4, 2024 Stephanie... Adam Zewe | MIT News Publication Date November 9, 2023 Press Inquiries Caption What do people mean when they say "generative AI," and why do these systems seem to be finding their way into practically every application imaginable? MIT AI experts help break down the ins and outs of

'The task is done.'